<a href="https://colab.research.google.com/github/Black-Kinetic/MindMage/blob/main/microgradClone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [724]:
import math
class value():
  def __init__(self,data,_children=[],_op=''):
    self.data=data
    self.grad=0
    self._prev=_children
    self._op=_op

  def __repr__(self):
    return f'value: {self.data}'

  def __add__(self,other):
    other = other if isinstance(other,value) else value(other)
    return value(self.data+other.data,[self,other],'+')

  def __mul__(self,other):
    other = other if isinstance(other,value) else value(other)
    return value(self.data*other.data,[self,other],'*')

  def __pow__(self,other):
    other = other if isinstance(other,value) else value(other)
    return value(self.data**other.data,[self,other],'**')

  def relu(self):
    a=self.data
    return value(a *(1 if a>0 else 0.1) ,[self,],'relu')

  def __radd__(self,other):
    other=value(other)
    return value(self.data+other.data,[self,other],'+')

  def __rmul__(self,other):
    other=value(other)
    return value(self.data*other.data,[self,other],'*')

  def __rpow__(self,other):
    other=value(other)
    return value(self.data**other.data,[self,other],'**')

  def exp(self):
    return value(math.exp(self.data),[self,],'exp')

  def __neg__(self):
    return self * -1

  def __sub__(self,other):
    return self+(-other)

  def __rsub__(self,other):
    return other+(-self)

  def __truediv__(self,other):
    return self*(other**-1)

  def __rtruediv__(self,other):
    return other*(self**-1)



In [725]:
import math
def bp(node):
  op=node._op
  n0=node._prev[0] if len(node._prev)>0 else None
  n1=node._prev[1] if len(node._prev)>1 else None
  for n in node._prev:
    if op == 'exp':
      n.grad+=node.grad*n.data.exp()
    if op == '+':
      n.grad+=node.grad
    if op == '*':
      if n == n0:
        n.grad += node.grad* n1.data
      else:
        n.grad += node.grad* n0.data
    if op == '**':
      if n == n0:
        n.grad += node.grad* n1.data * (n0.data**(n1.data-1))
      else:
        n.grad += node.grad* node.data * math.log(abs(n0.data))
    if op == '':
      return
    if op == 'relu':
      n.grad += node.grad * (1 if n.data > 0 else 0.1)
  for n in node._prev:
      bp(n)  # Only call once per child!

In [726]:
# from graphviz import Digraph

# def trace(root):
#   nodes, edges=set(),set()
#   def build(v):
#     if v not in nodes:
#       nodes.add(v)
#       for child in v._prev:
#         edges.add((child,v))
#         build(child)
#   build(root)
#   return nodes,edges

# def draw_dot(root):
#   dot=Digraph(format='svg',graph_attr={'rankdir': 'LR'})
#   root.grad=1
#   bp(root)
#   nodes,edges=trace(root)

#   for n in nodes:
#     uid=str(id(n))
#     dot.node(name=uid,label=f'data: {n.data} | grad: {n.grad}',shape='record')
#     if n._op:
#       dot.node(name=uid+n._op,label=n._op)
#       dot.edge(uid+n._op,uid)

#   for n1,n2 in edges:
#     dot.edge(str(id(n1)),str(id(n2))+n2._op)

#   return dot



In [728]:
import random
class neuron():
  def __init__(self,nin):
    self.w=[value(random.uniform(-1,1)) for _ in range(nin)]
    self.bias=value(random.uniform(-1,1))

  def __call__(self,x):
    act=sum(wi*xi for wi,xi in zip(self.w,x))+self.bias
    return act.relu()

  def parameters(self):
    return self.w+[self.bias]

class layer():
  def __init__(self,nin,nout):
    self.neurons=[neuron(nin) for _ in range(nout)]

  def __call__(self,x):
    outs=[n(x) for n in self.neurons]
    return outs[0] if len(outs) ==1 else outs

  def parameters(self):
    return [p for n in self.neurons for p in n.parameters()]

class mlp:
  def __init__(self, nin, nouts):
    sz=[nin]+nouts
    self.layers=[layer(sz[i],sz[i+1]) for i in range(len(nouts))]
  def __call__(self,x):
    for layer in self.layers:
      x=layer(x)
    return x

  def parameters(self):
     return [p for l in self.layers for p in l.parameters()]

In [729]:
xs=[
    [2,3,-1],
    [3,-1,0.5],
    [0.5,1,1],
    [1,1,-1],
]
ys=[1,-1,-1,1]

In [900]:
Amlan=mlp(3,[4,4,1])

In [905]:
from IPython.display import clear_output
while loss.data>0.001:
  ypred=[Amlan(x) for x in xs]
  clear_output()
  print(ypred)
  loss=sum([(yout-ygt)**2 for ygt,yout in zip(ys,ypred)])

  for p in Amlan.parameters():
      p.grad = 0

  loss.grad=1
  bp(loss)

  for p in Amlan.parameters():
    p.data+=-0.0001*p.grad

  print('loss',loss)
  Amlan.layers[0].neurons[0].w[0].grad
  if loss.data>10:
    break


[value: 0.9989488972059869, value: -1.0197341094394634, value: -0.9759697121876653, value: 1.0055617247315198]
loss value: 0.0009989274067851437


In [ ]:
from IPython.display import clear_output
clear_output()